In [51]:
import glob
import pandas as pd
import datetime
import os
from pathlib import Path
import urllib

#Global Variables
filepath = ' '


def main_process():
    """
    This function is to read the control file and to idfentify if
    1. School or Corp files
    2. Requistion or Application
    3. File Location
    Then pass the control to the appropiate functions
    """
    
    control_file = read_control_file('ControlData')
    control_file.index = ['Value']
    if control_file.loc['Value','RunType'] == 'SCHOOL' or 'CORP':
        if  control_file.loc['Value','Entity'] == 'APPL' or 'REQ':
            if control_file.loc['Value','RunType'] == 'SCHOOL':
                if control_file.loc['Value','Entity'] == 'APPL':
                    print ('SCHOOLAPPL')
                    
                    filepath = control_file.loc['Value','SchoolApplPath']
                    process_schl_appl_files(filepath)
                    
                elif control_file.loc['Value','Entity'] == 'REQ':
                    print('SCHOOL REQ')
                    
                    filepath = control_file.loc['Value','SchoolReqPath']
                    process_schl_req_files(filepath)
            else:
                if control_file.loc['Value','Entity'] == 'APPL':
                    print('CORPAPPL')
                    
                    filepath = control_file.loc['Value','CorpApplPath']
                    process_corp_appl_files(filepath)
                    
                elif control_file.loc['Value','Entity'] == 'REQ':
                    print('CORP REQ')
                    
                    filepath = control_file.loc['Value','CorpReqPath']
                    process_corp_req_files(filepath)
        else:
            print('Invalid Entity')
    else:
        print('Invalid Runtype')
        
def read_control_file(sheet_name):
    """
    *****ATTENTION - CHANGE THE CONTROL FILE LOCATION AS PER YOUR FOLDER IN YOUR SYSTEM*********
    Input - Sheet name of the control file
    
    Funtion:
    --------
    Read the control file sheet and return the data in the sheet
    """
    df_control_file = pd.read_excel('C:\Taleo\Control File.xlsx', sheet_name=sheet_name,header=1)
    return df_control_file

def write_output_file(basename,outdf):
    """
    Input - base name for the output file and the output dataframe.
    
    Funtion:
    --------
    1. Get the oupt file folder from the contril file.
    2. Get the current timestamp and apppned with the file name.
    3. Create the output file.
    """
    control_file = read_control_file('ControlData')
    control_file.index = ['Value']
    output_path = control_file.loc['Value','OutputPath'] # output file location
    filetime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")#Timestamp
    filename = basename + filetime+'.csv'
    output_path.replace('\\','/')
    outpath = os.path.join(output_path, filename)
    export_csv = outdf.to_csv (outpath, index = None, header=True)


def process_schl_appl_files(filepath):
    """
    Input - file path from control file
    
    Funtion:
    ---------
    1. Read all the excel file in the folder
    2. Clean each files.
    3. Create the applcation file
    """
    filepath.replace('\\','/') 
    os.chdir(filepath)
    combineddata = pd.DataFrame()
    
    for filename in glob.glob('*.xlsx'):
        taleofile = pd.read_excel(filename,header=None)
        taleofile = taleofile[3:]
        combineddata = pd.concat([combineddata,taleofile])
        print (filename)
        
    appl_schl_header = read_control_file('Appkschlheader') 
    combineddata.columns = appl_schl_header.columns
    write_output_file('SCHOOL_APPLICATION',combineddata)
    
    #Create candidate records from the application records.
    build_schl_candidate(combineddata)
    
        
def build_schl_candidate(df_application):
    """
    Input - Candidatate Applications combined data 
    
    Funtion: 
    --------
    1. Sort and remove duplicate candidates
    2. Create candidate file based on the mapping in the control file
    """
    #Sort dataframe based on cadidate id and application date
    df_application.sort_values(["Candidate_ID","Submission_Crtd_Dt"], ascending=False,inplace = True)
    #Remove duplicate candidate records and retain the one with lastest application date
    df_application.drop_duplicates(subset ="Candidate_ID", 
                     keep = 'first', inplace = True)
    candidate_mapng = read_control_file('SCApplToCandidate')
    
    #candidate_out = pd.DataFrame()
    candidate_header = candidate_mapng['candidate_fields'].tolist()
    candidate_out = pd.DataFrame(columns=candidate_header)
    
    for index, row in candidate_mapng.iterrows():
        target = row['candidate_fields']
        source = row['values']
        if row['mapping_type'] == 'M':
            candidate_out[target] = df_application[source].copy()
        else:
            if row['mapping_type'] == 'D':
                df_application.insert(0, target, source)
                candidate_out[target] = df_application[target].copy()
            else:
                print('Error*******Invalid mapping_type*******')
                
    write_output_file('SCHOOL_CANDIDATE',candidate_out)       


def process_corp_appl_files(filepath):
    """
    Input - file path from control file
    
    Funtion:
    ---------
    1. Read all the excel file in the folder
    2. Clean each files.
    3. Create the applcation file
    """
    filepath.replace('\\','/') 
    os.chdir(filepath)
    combineddata = pd.DataFrame()
    
    for filename in glob.glob('*.xlsx'):
        taleofile = pd.read_excel(filename,header=None)
        taleofile = taleofile[3:]
        combineddata = pd.concat([combineddata,taleofile])
        print (filename)
        
    appl_schl_header = read_control_file('Applcorpheader') 
    combineddata.columns = appl_schl_header.columns
    write_output_file('CORPORATE_APPLICATION',combineddata)
    
    #Create candidate records from the application records.
    build_corp_candidate(combineddata)
    
        
def build_corp_candidate(df_application):
    """
    Input - Candidatate Applications combined data 
    
    Funtion: 
    --------
    1. Sort and remove duplicate candidates
    2. Create candidate file based on the mapping in the control file
    """
    #Sort dataframe based on cadidate id and application date
    df_application.sort_values(["Candidate_ID","Submission_Crtd_Dt"], ascending=False,inplace = True)
    #Remove duplicate candidate records and retain the one with lastest application date
    df_application.drop_duplicates(subset ="Candidate_ID", 
                     keep = 'first', inplace = True)
    candidate_mapng = read_control_file('COApplToCandidate')
    
    #candidate_out = pd.DataFrame()
    candidate_header = candidate_mapng['candidate_fields'].tolist()
    candidate_out = pd.DataFrame(columns=candidate_header)
    
    for index, row in candidate_mapng.iterrows():
        target = row['candidate_fields']
        source = row['values']
        if row['mapping_type'] == 'M':
            candidate_out[target] = df_application[source].copy()
        else:
            if row['mapping_type'] == 'D':
                df_application.insert(0, target, source)
                candidate_out[target] = df_application[target].copy()
            else:
                print('Error*******Invalid mapping_type*******')
                
    write_output_file('CORPORATE_CANDIDATE',candidate_out)       
    
    

def process_schl_req_files(filepath):
    """
    Input - file path from control file
    
    Funtion:
    ---------
    1. Read all the excel file in the folder
    2. Clean each files.
    3. Create the school requistion file
    """
    filepath.replace('\\','/') 
    os.chdir(filepath)
    combineddata = pd.DataFrame()
    
    for filename in glob.glob('*.xlsx'):
        taleofile = pd.read_excel(filename,header=None)
        taleofile = taleofile[3:]
        combineddata = pd.concat([combineddata,taleofile])
        print (filename)
        
    appl_schl_header = read_control_file('Reqschlheader') 
    combineddata.columns = appl_schl_header.columns
    write_output_file('SCHOOL_REQUISITION',combineddata)




def process_corp_req_files(filepath):
    """
    Input - file path from control file
    
    Funtion:
    ---------
    1. Read all the excel file in the folder
    2. Clean each files.
    3. Create the corporate requisition file
    """
    filepath.replace('\\','/') 
    os.chdir(filepath)
    combineddata = pd.DataFrame()
    
    for filename in glob.glob('*.xlsx'):
        taleofile = pd.read_excel(filename,header=None)
        taleofile = taleofile[3:]
        combineddata = pd.concat([combineddata,taleofile])
        print (filename)
        
    appl_schl_header = read_control_file('Reqcorpheader') 
    combineddata.columns = appl_schl_header.columns
    write_output_file('CORPORATE_REQUISITION',combineddata)


    
main_process()
        

CORPAPPL
Candidate Corporate Report 2019 Apr - Jun.xlsx
Candidate Corporate Report 2019 Jan-Mar.xlsx
Candidate Corporate Report 2019 July- Sep.xlsx
Candidate Corporate Report 2019 Oct-Dec.xlsx
